<a href="https://colab.research.google.com/github/ManuelBagasina/solarspell/blob/main/cleaning_surveys.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

## Install Libraries

In [ ]:
# !pip install numpy pandas
import numpy as np
import re
import pandas as pd

## Load csv

In [ ]:
# 1. Load
df = pd.read_csv('sudan_survey.csv')

# Cleaning Data

In [ ]:
# 2. Define mapping (all keys lowercased)
likert_fuzzy_map = {
    # Strongly Agree
    "significantly better":      "STRONGLY AGREE",
    "very improved":             "STRONGLY AGREE",
    "very motivated":            "STRONGLY AGREE",
    "very useful":               "STRONGLY AGREE",
    "completely addresses gaps": "STRONGLY AGREE",
    "very comfortable":          "STRONGLY AGREE",

    # Agree
    "improved":                  "AGREE",
    "motivated":                 "AGREE",
    "useful":                    "AGREE",
    "addresses gaps":            "AGREE",
    "comfortable":               "AGREE",

    # Neutral
    "somewhat improved":         "NEUTRAL",
    "somewhat motivated":        "NEUTRAL",
    "somewhat useful":           "NEUTRAL",
    "slightly improved":         "NEUTRAL",
    "partially addresses gaps":  "NEUTRAL",
    "somewhat comfortable":      "NEUTRAL",
    "no change":                 "NEUTRAL",
    "no changes":                "NEUTRAL",

    # Disagree
    "not motivated":             "DISAGREE",
    "not useful":                "DISAGREE",
    "not comfortable":           "DISAGREE",
    "not better":                "DISAGREE",
    "does not address gaps":     "DISAGREE",

    # Strongly Disagree
    "worse":                     "STRONGLY DISAGREE",
    "not at all useful":         "STRONGLY DISAGREE",
    "very discomfortable":       "STRONGLY DISAGREE",
    "not at all motivated":      "STRONGLY DISAGREE",

    # Missing values
    "n/a":                       None,
    "":                          None
}

In [ ]:
# 3. Exclude non-Likert columns (from form review)
excluded_cols = [
    "(IF YES IN-PERSON) WHY DID YOU CHOOSE THIS LEVEL OF USEFULNESS?",
    "(IF YES OFFLINE TRAINING COURSE) WHY DID YOU CHOOSE THIS LEVEL OF USEFULNESS?",
    "DO YOU USE THE SOLARSPELL LIBRARY TO TEACH ABOUT CLIMATE CHANGE & SUSTAINABILITY?",
    "DO YOU USE THE SOLARSPELL LIBRARY TO TEACH ABOUT GENDER EQUITY AND INCLUSION?",
    "DO YOU USE THE SOLARSPELL LIBRARY TO TEACH ABOUT INFORMATION LITERACY?",
    "DO YOU USE THE SOLARSPELL LIBRARY TO TEACH ABOUT DIGITAL LITERACY?",
    "FOR QUESTIONS 31-33 (IMPROVED/BETTER SKILLS), PLEASE DESCRIBE WHY YOU CHOSE THESE LEVELS",
    "PLEASE PROVIDE SOME EXAMPLES OF THE MOST POPULAR OR USEFUL CONTENT ON THE SOLARSPELL LIBRARY",
    "DESCRIBE WHY YOU CHOSE THIS LEVEL",
    "WHY DID YOU CHOOSE THESE LEVELS FOR QUESTIONS 17-20 (MOTIVATION AND PREPAREDNESS)",
    "IF YOU ANSWERED YES FOR QUESTIONS 26-29 (\"DO YOU USE SOLARSPELL TO TEACH ABOUT...\") PLEASE SHARE MORE INFORMATION (FOR EXAMPLE: AN EXAMPLE OF HOW THE RESOURCE IS USED? THE MOST POPULAR CONTENT? VIDEOS OR WORKSHEETS? ANY INFO WELCOME)",
    "PLEASE PROVIDE ANY ADDITIONAL INFORMATION TO SUPPORT YOUR RESPONSES TO QUESTIONS 40-45 (RE: STUDENTS)",
    "ADDITIONAL THOUGHTS, COMMENTS, AND/OR QUESTIONS"
]

In [ ]:
# 4. Keywords to identify Likert-style questions
keywords = [
    "improved", "motivation", "motivat", "useful", "comfortable", "prepared",
    "better", "address", "confidence", "extent"
]

In [ ]:
# 5. Build Likert columns list
likert_cols = [
    col for col in df.columns
    if any(k in col.lower() for k in keywords) and col not in excluded_cols
]

In [ ]:
# 6. Define the cleaning function
def clean_likert_cell(val):
    if isinstance(val, str):
        norm = val.strip().lower()
        for phrase, label in likert_fuzzy_map.items():
            if phrase in norm:
                return label
    return val  # unchanged

In [ ]:
# 7. Apply mapping
for col in likert_cols:
    df[col] = df[col].apply(clean_likert_cell)

In [ ]:
# 8. Save cleaned dataset
df.to_csv("South_Sudan_Survey_CLEANED_FINAL.csv", index=False)
print("✅ Cleaned Likert responses saved to South_Sudan_Survey_CLEANED_FINAL.csv")

✅ Cleaned Likert responses saved to South_Sudan_Survey_CLEANED_FINAL.csv


In [ ]:
# 9. Audit: Check for unexpected values
print("\n🔎 Unmapped Values by Column:")
for col in likert_cols:
    unique_vals = [v for v in df[col].dropna().unique()
                   if v not in ["STRONGLY AGREE", "AGREE", "NEUTRAL", "DISAGREE", "STRONGLY DISAGREE"]]
    if unique_vals:
        print(f"⚠️ {col}: {unique_vals}")
    else:
        print(f"✅ {col}: all values mapped")


🔎 Unmapped Values by Column:
✅ (IF YES IN-PERSON) HOW USEFUL WAS THE TRAINING YOU RECEIVED ON THE SOLARSPELL LIBRARY: all values mapped
✅ (IF YES OFFLINE TRAINING COURSE) HOW USEFUL WAS THE TRAINING YOU RECEIVED ON THE SOLARSPELL LIBRARY: all values mapped
✅ HOW MOTIVATED ARE YOU TO USE THE SOLARSPELL LIBRARY?: all values mapped
✅ TO WHAT EXTENT HAS USING THE SOLARSPELL AFFECTED YOUR LEVEL OF MOTIVATION AS A TEACHER: all values mapped
✅ TO WHAT EXTENT HAS USING THE SOLARSPELL MADE YOU FEEL MORE PREPARED OR ABLE TO MORE EFFECTIVELY DO YOUR JOB?: all values mapped
✅ AS A TEACHER, TO WHAT EXTEND DOES THE SOLARSPELL LIBRARY ADDRESS THE RESOURCE GAPS THAT YOU HAVE?: all values mapped
✅ SINCE USING SOLARSPELL, ARE YOU BETTER ABLE TO USE OTHER SIMILAR FORMS OF TECHNOLOGY (LIKE THE INTERNET, TABLETS, SMARTPHONES, COMPUTERS)?: all values mapped
✅ SINCE USING SOLARSPELL, ARE YOU BETTER ABLE TO CARRY OUT RESEARCH OR FIND NEW INFORMATION?: all values mapped
✅ IN GENERAL, HOW COMFORTABLE ARE STUDE